In [3]:
import pandas as pd
import nfl_data_py as nfl

In [2]:
df = pd.read_pickle('2013-2023_plays.pkl')

In [6]:
df_fg = df[df['field_goal_attempt']==1]

In [17]:
# df_fg['play_duration'] = 
df_fg[['time','drive_game_clock_end']]

df_fg['drive_game_clock_end'] = pd.to_datetime(df_fg['drive_game_clock_end'], format='%M:%S')
df_fg['time'] = pd.to_datetime(df_fg['time'], format='%M:%S')


df_fg['play_time'] = (df_fg['time']-df_fg['drive_game_clock_end']).dt.seconds

df_fg['play_time'].mode()


/var/folders/v5/23qsn2tj1pg_yz86qk9f46b40000gp/T/ipykernel_83860/2581555462.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fg['drive_game_clock_end'] = pd.to_datetime(df_fg['drive_game_clock_end'], format='%M:%S')
/var/folders/v5/23qsn2tj1pg_yz86qk9f46b40000gp/T/ipykernel_83860/2581555462.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fg['time'] = pd.to_datetime(df_fg['time'], format='%M:%S')
/var/folders/v5/23qsn2tj1pg_yz86qk9f46b40000gp/T/ipykernel_83860/2581555462.py:8: SettingWithCopyWar

0    4
Name: play_time, dtype: int32

In [23]:
class GameState:
    def __init__(self, **kwargs):
        self.receive_2h_ko = self._validate_receive_2h_ko(kwargs.get('receive_2h_ko', 0))
        self.home_team = self._validate_team(kwargs.get('home_team', ""))
        self.posteam = self._validate_team(kwargs.get('posteam', ""))
        self.score_differential = self._validate_score_differential(kwargs.get('score_differential', 0))
        self.half_seconds_remaining = self._validate_seconds(kwargs.get('half_seconds_remaining', 1800))
        self.game_seconds_remaining = self._validate_seconds(kwargs.get('game_seconds_remaining', 3600))
        self.spread_line = self._validate_spread_line(kwargs.get('spread_line', [1, 3, 4, 7, 14]))
        self.down = self._validate_down(kwargs.get('down', 1))
        self.ydstogo = self._validate_ydstogo(kwargs.get('ydstogo', 10))
        self.yardline_100 = self._validate_yardline_100(kwargs.get('yardline_100', 75))
        self.posteam_timeouts_remaining = self._validate_timeouts(kwargs.get('posteam_timeouts_remaining', 3))
        self.defteam_timeouts_remaining = self._validate_timeouts(kwargs.get('defteam_timeouts_remaining', 3))
        self.goal_to_go = self._validate_goal_to_go(kwargs.get('goal_to_go',0))

    def update_attribute(self, attribute, value):
        if hasattr(self, attribute):
            validation_method = getattr(self, f'_validate_{attribute}')
            setattr(self, attribute, validation_method(value))
        else:
            raise AttributeError(f"GameState has no attribute '{attribute}'")

    def _validate_receive_2h_ko(self, value):
        if value not in [0, 1]:
            raise ValueError("receive_2h_ko must be either 0 or 1")
        return value

    def _validate_team(self, value):
        if not isinstance(value, str) or len(value) < 1 or len(value) > 4:
            raise ValueError("home_team and posteam must be a string with length between 1 and 4")
        return value

    def _validate_score_differential(self, value):
        if not isinstance(value, int):
            raise ValueError("score_differential must be an integer")
        return value

    def _validate_seconds(self, value):
        if not isinstance(value, int) or value < 0:
            raise ValueError("half_seconds_remaining and game_seconds_remaining must be non-negative integers")
        return value

    def _validate_spread_line(self, value):
        if not isinstance(value, int):
            raise ValueError("spread_line must be an integer")
        return value

    def _validate_down(self, value):
        if not isinstance(value, int) or value < 1 or value > 4:
            raise ValueError("down must be an integer between 1 and 4")
        return value

    def _validate_ydstogo(self, value):
        if not isinstance(value, int) or value < 1 or value > 99:
            raise ValueError("ydstogo must be an integer between 1 and 99")
        return value

    def _validate_yardline_100(self, value):
        if not isinstance(value, int) or value < 1 or value > 99:
            raise ValueError("yardline_100 must be an integer between 1 and 99")
        return value

    def _validate_timeouts(self, value):
        if not isinstance(value, int) or value < 0 or value > 3:
            raise ValueError("posteam_timeouts_remaining and defteam_timeouts_remaining must be integers between 0 and 3")
        return value

    def _validate_goal_to_go (self, value):
        if value not in [0, 1]:
            raise ValueError("goal_to_go must be either 0 or 1")
        return value



    def __str__(self):
        return f"GameState(receive_2h_ko={self.receive_2h_ko}, home_team='{self.home_team}', " \
               f"posteam='{self.posteam}', score_differential={self.score_differential}, " \
               f"half_seconds_remaining={self.half_seconds_remaining}, " \
               f"game_seconds_remaining={self.game_seconds_remaining}, spread_line={self.spread_line}, " \
               f"down={self.down}, ydstogo={self.ydstogo}, yardline_100={self.yardline_100}, " \
               f"posteam_timeouts_remaining={self.posteam_timeouts_remaining}, " \
               f"defteam_timeouts_remaining={self.defteam_timeouts_remaining})"

In [24]:
def fg_made(game_state):
    if game_state.half_seconds_remaining <5:
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining - game_state.half_seconds_remaining)
        game_state.update_attribute('half_seconds_remaining',0)
    else:
        game_state.update_attribute('half_seconds_remaining',game_state.half_seconds_remaining-5)
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining-5)

    game_state.update_attribute('score_differential',-(game_state.score_differential+3))

    if game_state.home_team == game_state.posteam:
        game_state.update_attribute('posteam','xxx')
    else:
        game_state.update_attribute('posteam',game_state.home_team)
        
    game_state.update_attribute('yardline_100',75)
    game_state.update_attribute('down',1)
    game_state.update_attribute('ydstogo',10)

    game_state.update_attribute('goal_to_go',0)


In [25]:
def fg_miss(game_state):
    if game_state.half_seconds_remaining <5:
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining - game_state.half_seconds_remaining)
        game_state.update_attribute('half_seconds_remaining',0)
    else:
        game_state.update_attribute('half_seconds_remaining',game_state.half_seconds_remaining-5)
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining-5)


    if game_state.home_team == game_state.posteam:
        game_state.update_attribute('posteam','xxx')
    else:
        game_state.update_attribute('posteam',game_state.home_team)
        
    game_state.update_attribute('yardline_100',100-(game_state.yardline_100+7))
    game_state.update_attribute('down',1)
    game_state.update_attribute('ydstogo',10)

    game_state.update_attribute('goal_to_go',0)


In [26]:
import pickle
def punt(game_state):
    if game_state.half_seconds_remaining <5:
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining - game_state.half_seconds_remaining)
        game_state.update_attribute('half_seconds_remaining',0)
    else:
        game_state.update_attribute('half_seconds_remaining',game_state.half_seconds_remaining-5)
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining-5)

    if game_state.home_team == game_state.posteam:
        game_state.update_attribute('posteam','xxx')
    else:
        game_state.update_attribute('posteam',game_state.home_team)
    
    with open('punt_yardage_model.pkl', 'rb') as file:
        # Punt Model predicts new yard line given current yard line
        punt_model = pickle.load(file)

    game_state.update_attribute('yardline_100',punt_model(game_state.yardline_100))
    game_state.update_attribute('down',1)
    game_state.update_attribute('ydstogo',10)

In [29]:
def go_for_it_fail(game_state):
    if game_state.half_seconds_remaining <5:
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining - game_state.half_seconds_remaining)
        game_state.update_attribute('half_seconds_remaining',0)
    else:
        game_state.update_attribute('half_seconds_remaining',game_state.half_seconds_remaining-5)
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining-5)

    if game_state.home_team == game_state.posteam:
        game_state.update_attribute('posteam','xxx')
    else:
        game_state.update_attribute('posteam',game_state.home_team)

    with open('GFI_fail_model.pkl.pkl', 'rb') as file:
    # GFI Model predicts  yard gain/loss given current game state
        GFI_fail_model = pickle.load(file)
    
    game_state.update_attribute('yardline_100',100-(game_state.yardline_100 - GFI_fail_model(game_state.yardline_100,game_state.ydstogo,game_state.goal_to_go)))

    game_state.update_attribute('down',1)
    game_state.update_attribute('ydstogo',10)


In [30]:

def go_for_it_success(game_state):
    if game_state.half_seconds_remaining <5:
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining - game_state.half_seconds_remaining)
        game_state.update_attribute('half_seconds_remaining',0)
    else:
        game_state.update_attribute('half_seconds_remaining',game_state.half_seconds_remaining-5)
        game_state.update_attribute('game_seconds_remaining',game_state.game_seconds_remaining-5)


    if game_state.goal_to_go ==1:
        game_state.update_attribute('score_differential',-(game_state.score_differential+7))
        # New yardage after kickoff
        game_state.update_attribute('yardline_100',75)
        game_state.update_attribute('down',1)
        game_state.update_attribute('ydstogo',10)
        game_state.update_attribute('goal_to_go',0)
    else:
        with open('GFI_success.pkl', 'rb') as file:
            # GFI Model predicts  yard gain/loss given current game state
            GFI_success_model = pickle.load(file)
        game_state.update_attribute('yardline_100',(game_state.yardline_100 + GFI_success_model(game_state.yardline_100,game_state.ydstogo)))
        
    
        

In [31]:
# df[['score_differential','home_team','posteam','half_seconds_remaining','game_seconds_remaining','spread_line','down','ydstogo','yardline_100',
#     'posteam_timeouts_remaining','defteam_timeouts_remaining','wp']]